# 🧠 Data Challenge 2025 – Gas Detection

This notebook presents the solution developed for the **Data Challenge 2025 (ENS / Bertin Technologies)**.  
The objective of the challenge is to **predict the alarm level of 23 gases** based on multivariate sensor measurements.

The final model relies on:
- **Extra Trees Regressors** trained on several variants (with and without *row-wise* features),  
- an **optimized blending** of predictions (both global and per-target),  
- a **linear calibration** and **shrinkage toward the mean** to stabilize outputs.

This notebook is designed to:
- preprocess the data,  
- train the models,  
- and **automatically generate the final submission** in the format required by the challenge platform.

Performance is evaluated using the **Weighted RMSE**, a weighted root mean squared error metric.


## 📦 1. Imports & Data Loading  
Read the files `x_train.csv`, `y_train.csv`, and `x_test.csv` from the `../DATA` directory.  
Display basic dimensions for a quick sanity check.


In [ ]:

from __future__ import annotations
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Tuple
import time

import numpy as np
import pandas as pd

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.linear_model import LinearRegression

# Data import
DATA_DIR = Path("../DATA").resolve()
assert DATA_DIR.exists(), f"Le dossier DATA est introuvable: {DATA_DIR}"

X_train = pd.read_csv(DATA_DIR / "x_train.csv")
y_train = pd.read_csv(DATA_DIR / "y_train.csv")
X_test  = pd.read_csv(DATA_DIR / "x_test.csv")

print(f"✅ Données chargées :")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}")


## ⚙️ 2. Utility Functions  
Reusable functions used throughout the pipeline: custom metric (Weighted RMSE), stratified folds,  
row-wise cleaning and feature generation, linear calibration, and shrinkage toward the mean.


In [ ]:


def weighted_rmse(y_true: np.ndarray, y_pred: np.ndarray) -> float: # Challenge métric
    y_pred = np.clip(y_pred, 0.0, 1.0)
    w = np.where(y_true >= 0.5, 1.2, 1.0)
    mse_per_sample = np.mean(w * (y_pred - y_true)**2, axis=1)
    return float(np.sqrt(np.mean(mse_per_sample)))


def make_stratified_folds(y: pd.DataFrame, n_splits=5, random_state=42): # Stratified folds based on target mean (for stable out-of-fold performance)
    y_mean = y.mean(axis=1).to_numpy()
    ranks = pd.Series(y_mean).rank(method="average", pct=True).to_numpy()
    nbins = max(2, min(10, len(y)//n_splits))
    bins = np.floor(ranks * nbins).astype(int)
    bins[bins == nbins] = nbins - 1
    if len(np.unique(bins)) < 2:
        print("⚠️ Stratification impossible (bins uniques) → fallback KFold.")
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        return list(kf.split(np.zeros(len(y))))
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    return list(skf.split(np.zeros(len(y)), bins))


# Clip by quantiles (fitted on train, applied to test) to reduce the influence of outliers
# Drop 'Humidity' feature (too noisy according to EDA)
def clean_features(X_train, X_test, lower_q=0.01, upper_q=0.99, drop_humidity=True):
    Xtr = X_train.copy(); Xte = X_test.copy()
    for col in Xtr.columns:
        if col != "ID" and pd.api.types.is_numeric_dtype(Xtr[col]):
            low = Xtr[col].quantile(lower_q)
            high = Xtr[col].quantile(upper_q)
            Xtr[col] = Xtr[col].clip(low, high)
            Xte[col] = Xte[col].clip(low, high)
    if drop_humidity and "Humidity" in Xtr.columns:
        Xtr = Xtr.drop(columns=["Humidity"]) ; Xte = Xte.drop(columns=["Humidity"]) 
    return Xtr, Xte


# Add robust "row-level" features (means/percentiles/spreads) to capture global sensor structure
def add_rowwise_features(X: pd.DataFrame,
                         exclude_cols=("ID",),
                         prefix="rw_") -> pd.DataFrame:
    Xo = X.copy()
    num_cols = [c for c in Xo.columns if c not in exclude_cols and pd.api.types.is_numeric_dtype(Xo[c])]
    if len(num_cols) == 0:
        return Xo
    V = Xo[num_cols].to_numpy(dtype=float)

    def row_nanpercentile(a, p):
        return np.nanpercentile(a, p, axis=1)

    mean   = np.nanmean(V, axis=1)
    median = np.nanmedian(V, axis=1)
    std    = np.nanstd(V, axis=1)
    vmin   = np.nanmin(V, axis=1)
    vmax   = np.nanmax(V, axis=1)
    rng    = vmax - vmin

    q25 = row_nanpercentile(V, 25)
    q75 = row_nanpercentile(V, 75)
    iqr = q75 - q25

    abs_dev = np.abs(V - median[:, None])
    mad = np.nanmedian(abs_dev, axis=1)

    l1 = np.nansum(np.abs(V), axis=1)
    l2 = np.sqrt(np.nansum(V * V, axis=1))

    n_nan   = np.isnan(V).sum(axis=1).astype(float)
    nan_rat = n_nan / V.shape[1]

# Robust row-wise aggregates (nan-safe): useful when sensor distributions differ
    R = pd.DataFrame({
        f"{prefix}mean": mean,
        f"{prefix}median": median,
        f"{prefix}std": std,
        f"{prefix}min": vmin,
        f"{prefix}max": vmax,
        f"{prefix}range": rng,
        f"{prefix}q25": q25,
        f"{prefix}q75": q75,
        f"{prefix}iqr": iqr,
        f"{prefix}mad": mad,
        f"{prefix}l1": l1,
        f"{prefix}l2": l2,
        f"{prefix}n_nan": n_nan,
        f"{prefix}nan_ratio": nan_rat,
    }, index=Xo.index)

    return pd.concat([Xo, R], axis=1)


# Independent linear calibration per target: corrects scale bias between OOF and y_true
# If variance is near zero → fallback to target mean (avoids unstable solutions)
def calibrate_predictions_linear(y_true_df, oof_pred_df, test_pred_df, eps=1e-9):
    y_true = y_true_df.values
    y_oof  = oof_pred_df.values
    y_te   = test_pred_df.values.copy()
    for j in range(y_true.shape[1]):
        x = y_oof[:, [j]]
        if np.std(x) < eps:
            a, b = 0.0, float(y_true[:, j].mean())
            y_oof[:, j] = a * x.ravel() + b
            y_te[:,  j] = a * y_te[:,  j] + b
        else:
            lr = LinearRegression()
            lr.fit(x, y_true[:, j])
            y_oof[:, j] = lr.predict(x)
            y_te[:,  j] = lr.predict(y_te[:, [j]])
    y_oof = np.clip(y_oof, 0.0, 1.0)
    y_te  = np.clip(y_te,  0.0, 1.0)
    return pd.DataFrame(y_oof, columns=y_true_df.columns), pd.DataFrame(y_te, columns=test_pred_df.columns)

# Global shrinkage toward the training mean


def shrink_to_mean(test_pred_df, y_train_df, alpha=0.95):
    mu = y_train_df.mean(axis=0).values
    Y  = test_pred_df.values
    Ys = alpha * Y + (1 - alpha) * mu
    return pd.DataFrame(np.clip(Ys, 0.0, 1.0), columns=test_pred_df.columns)

# Target-wise shrinkage with distinct alpha_j values (allows adjusting shrinkage strength per column)

def shrink_to_mean_per_target(pred_df: pd.DataFrame, y_train_df: pd.DataFrame, alphas: np.ndarray) -> pd.DataFrame:
    """alphas: array shape (n_targets,) avec alpha_j pour chaque cible."""
    mu = y_train_df.mean(axis=0).to_numpy()          # (T,)
    Y  = pred_df.to_numpy()                           # (N,T)
    A  = np.asarray(alphas).reshape(1, -1)            # (1,T)
    Ys = A * Y + (1.0 - A) * mu.reshape(1, -1)
    return pd.DataFrame(np.clip(Ys, 0.0, 1.0), columns=pred_df.columns)


## 🌲 3. ExtraTrees Variants  
Definition of a small family of ExtraTrees models (with and without *row-wise* features),  
using different random seeds and settings (depth, bootstrap).  
Default parameters are centralized, and each variant only specifies its overrides.


In [ ]:

# Configuration d'une variante ExtraTrees (nom, usage des features row-wise, overrides de params sklearn)
@dataclass
class ETConfig:
    name: str
    use_rw: bool
    params: Dict


def make_et(**kw) -> ExtraTreesRegressor:
    return ExtraTreesRegressor(
        n_estimators=kw.get("n_estimators", 500),
        max_features=kw.get("max_features", 0.7),
        min_samples_split=kw.get("min_samples_split", 8),
        min_samples_leaf=kw.get("min_samples_leaf", 2),
        max_depth=kw.get("max_depth", None),
        bootstrap=kw.get("bootstrap", False),
        max_samples=kw.get("max_samples", None),
        n_jobs=-1,
        random_state=kw.get("random_state", 42),
    )

VARIANTS: List[ETConfig] = [
    ETConfig("ET-Base", use_rw=False, params=dict(random_state=42)),
    ETConfig("ET-RW",   use_rw=True,  params=dict(random_state=42)),
    ETConfig("ET-DepthCap",   use_rw=False, params=dict(random_state=42, max_depth=24, max_features=0.8, min_samples_leaf=1)),
    ETConfig("ET-Bootstrap",    use_rw=True,  params=dict(random_state=42, bootstrap=True, max_samples=0.8, max_features=0.6, min_samples_leaf=3)),
    ETConfig("ET-Seed13",     use_rw=False, params=dict(random_state=13)),
    ETConfig("ET-Seed71",       use_rw=True,  params=dict(random_state=71)),
]


In [ ]:


def run_et_variant(
    cfg: ETConfig,
    X_base_tr: pd.DataFrame,
    X_base_te: pd.DataFrame,
    X_rw_tr: pd.DataFrame,
    X_rw_te: pd.DataFrame,
    y_fit: pd.DataFrame,
    n_splits: int = 5,
    seed: int = 42,
) -> Tuple[np.ndarray, np.ndarray, float]:
    
   # Stratified folds based on target mean
    folds = make_stratified_folds(y_fit, n_splits=n_splits, random_state=seed)

    # Feature selection based on variant (base vs row-wise); drop 'ID' if present
    Xtr = (X_rw_tr if cfg.use_rw else X_base_tr).drop(columns=["ID"], errors="ignore")
    Xte = (X_rw_te if cfg.use_rw else X_base_te).drop(columns=["ID"], errors="ignore")


    oof = np.zeros((len(Xtr), y_fit.shape[1]), dtype=float)
    te  = np.zeros((len(Xte), y_fit.shape[1]), dtype=float)



    t0 = time.time()

    for i, (tr_idx, va_idx) in enumerate(folds, 1):
        print(f"🚀 {cfg.name} — Fold {i}/{n_splits}")
        X_tr, X_va = Xtr.iloc[tr_idx], Xtr.iloc[va_idx]
        y_tr, y_va = y_fit.iloc[tr_idx], y_fit.iloc[va_idx]

        model = MultiOutputRegressor(make_et(**cfg.params), n_jobs=1)
        model.fit(X_tr, y_tr)

        oof[va_idx] = model.predict(X_va)
        te += model.predict(Xte) / n_splits


    elapsed = time.time() - t0

    # OOF score using the challenge's evaluation metric

    score = weighted_rmse(y_fit.values, oof)
    print(f"🎯 {cfg.name} — OOF Weighted RMSE: {score:.6f} | ⏱ {elapsed:.1f}s\n")



    return oof, te, score

## 🧩 4. Model Blending  
Optimization of weights on the Dirichlet simplex to combine out-of-fold (OOF) predictions.


In [ ]:

# Sample weights (n_samples, k) on the simplex using a Dirichlet distribution
def _dirichlet_weights(k: int, n_samples: int = 4000, temperature: float = 1.0, rng: np.random.Generator | None = None):
    rng = np.random.default_rng() if rng is None else rng
    alpha = np.ones(k) * temperature
    return rng.dirichlet(alpha, size=n_samples)  # (n_samples, k)


def optimize_global_weights(oofs: List[np.ndarray], y_true: np.ndarray, n_samples: int = 6000) -> np.ndarray:
    k = len(oofs)
    W = _dirichlet_weights(k, n_samples=n_samples)
    oofs_stack = np.stack(oofs, axis=-1)  # (n_samples, n_targets, k)
    best_w = None
    best_score = float("inf")
    for w in W:
        y_blend = np.tensordot(oofs_stack, w, axes=([2],[0]))  # (n_samples, n_targets)
        score = weighted_rmse(y_true, y_blend)
        if score < best_score:
            best_score = score
            best_w = w
    print(f"✅ Best global blend (OOF): {best_score:.6f} | weights={np.round(best_w,3)}")
    return best_w


def optimize_per_target_weights(oofs: List[np.ndarray], y_true: np.ndarray, n_samples: int = 3000) -> np.ndarray:
    k = len(oofs)
    T = y_true.shape[1]
    oofs_stack = np.stack(oofs, axis=-1)  # (n_samples, n_targets, k)
    W_out = np.zeros((T, k), dtype=float)
    for j in range(T):
        W = _dirichlet_weights(k, n_samples=n_samples)
        best_w = None
        best = float("inf")
        yj_true = y_true[:, j]
        oofs_j = oofs_stack[:, j, :]  # (n_samples, k)
        for w in W:
            yj_pred = oofs_j @ w
            yj_pred = np.clip(yj_pred, 0, 1)
            wgt = np.where(yj_true >= 0.5, 1.2, 1.0)
            mse = np.mean(wgt * (yj_pred - yj_true) ** 2)
            rmse = np.sqrt(mse)
            if rmse < best:
                best = rmse
                best_w = w
        W_out[j] = best_w
    return W_out


def apply_global_weights(oofs: List[np.ndarray], tests: List[np.ndarray], w: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    oof_stack = np.stack(oofs, axis=-1)
    test_stack = np.stack(tests, axis=-1)
    oof_blend = np.tensordot(oof_stack, w, axes=([2],[0]))
    test_blend = np.tensordot(test_stack, w, axes=([2],[0]))
    return np.clip(oof_blend, 0, 1), np.clip(test_blend, 0, 1)


def apply_per_target_weights(oofs: List[np.ndarray], tests: List[np.ndarray], Wt: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    oof_stack = np.stack(oofs, axis=-1)
    test_stack = np.stack(tests, axis=-1)
    oof_blend = np.einsum('ntk,tk->nt', oof_stack, Wt)
    test_blend = np.einsum('mtk,tk->mt', test_stack, Wt)
    return np.clip(oof_blend, 0, 1), np.clip(test_blend, 0, 1)





## 🧪 5. Pipeline: diversity + blending + calibration + shrinkage  
All model variants are trained, blend weights are optimized on OOF predictions (globally and per target),  
then a linear calibration and shrinkage toward the training mean are applied.  
The pipeline returns the final OOF/Test predictions along with metadata (weights, alpha, scores).


In [ ]:


def run_step1_diversity_blend(cv_splits: int = 5, seed: int = 42, do_per_target_weights: bool = True,
                              shrink_grid=(0.92,0.95,0.97,0.99)):
    """Entraîne les variantes ET, optimise le blend sur OOF, applique calibration + shrink,
                                retourne les résultats."""


   # 1) Cleaning & feature engineering
    X_base_tr, X_base_te = clean_features(X_train, X_test, drop_humidity=True)
    X_rw_tr  = add_rowwise_features(X_base_tr, exclude_cols=("ID",))
    X_rw_te  = add_rowwise_features(X_base_te, exclude_cols=("ID",))

   # 2) Targets: remove 'ID' and 'c15'
    y_fit = y_train.drop(columns=["ID", "c15"], errors="ignore").copy()
    y_fit = y_fit.loc[X_base_tr.index]

    # 3) Train model variants
    oofs: List[np.ndarray] = []
    tests: List[np.ndarray] = []
    scores: List[Tuple[str, float]] = []
    for cfg in VARIANTS:
        oof, te, sc = run_et_variant(cfg, X_base_tr, X_base_te, X_rw_tr, X_rw_te, y_fit, n_splits=cv_splits, seed=seed)
        oofs.append(oof); tests.append(te); scores.append((cfg.name, sc))

    print("=== Résumé variantes (OOF) ===")
    for name, sc in scores:
        print(f"{name:<24} : {sc:.6f}")

    # 4) Blends
    y_true = y_fit.values
    w_glob = optimize_global_weights(oofs, y_true, n_samples=6000)
    oof_g, test_g = apply_global_weights(oofs, tests, w_glob)
    score_g = weighted_rmse(y_true, oof_g)
    print(f"🏁 Blend GLOBAL — OOF: {score_g:.6f}")

    blends = {"global": (oof_g, test_g, w_glob, score_g)}

    if do_per_target_weights:
        Wt = optimize_per_target_weights(oofs, y_true, n_samples=3000)
        oof_t, test_t = apply_per_target_weights(oofs, tests, Wt)
        score_t = weighted_rmse(y_true, oof_t)
        print(f"🏁 Blend PAR CIBLE — OOF: {score_t:.6f}")
        blends["per_target"] = (oof_t, test_t, Wt, score_t)

    # 5) Calibration + Shrink
    results = {}
    for k, (oof_b, test_b, W, sc_b) in blends.items():
        oof_df  = pd.DataFrame(oof_b,  columns=y_fit.columns)
        test_df = pd.DataFrame(test_b, columns=y_fit.columns)
        oof_cal, test_cal = calibrate_predictions_linear(y_fit, oof_df, test_df)

        best_alpha, best_score = None, float("inf")
        for a in shrink_grid:
            tmp = shrink_to_mean(oof_cal, y_fit, alpha=a)
            s = weighted_rmse(y_true, tmp.values)
            if s < best_score:
                best_alpha, best_score = a, s
        print(f"🔧 Blend={k}: meilleur alpha={best_alpha} → OOF après calib+shrink: {best_score:.6f}")

        oof_final  = shrink_to_mean(oof_cal,  y_fit, alpha=best_alpha).values
        test_final = shrink_to_mean(test_cal, y_fit, alpha=best_alpha).values
        oof_final  = np.clip(oof_final,  0, 1)
        test_final = np.clip(test_final, 0, 1)

        results[k] = {
            "weights": W,
            "oof_pre_calib": sc_b,
            "oof_post": best_score,
            "alpha": best_alpha,
            "oof_preds": oof_final,
            "test_preds": test_final,
        }



        # 6) Final selection
    best_key = min(results.keys(), key=lambda k: results[k]["oof_post"])
    print(f"✅ Sélection finale: {best_key} — OOF={results[best_key]['oof_post']:.6f} (alpha={results[best_key]['alpha']})")


    return {
        "variant_scores": scores,
        "blends": results,
        "best": best_key,
        "y_columns": y_fit.columns.tolist(),
    }




## 🏁 6. Pipeline Execution & Submission Generation  
The full pipeline is executed (variants → blending → calibration → shrinkage),  
the best blend is selected, and the final submission file is generated.


In [ ]:


# Run the pipeline (CV=3 for demo; use 5 for final run)
out = run_step1_diversity_blend(cv_splits=3, seed=42, do_per_target_weights=True)

# Retrieve the key of the best blend and its test predictions
best_key = out["best"]
y_pred_blend = out["blends"][best_key]["test_preds"]  # (n_test, n_targets_sans_c15)



In [ ]:

# Target columns: exclude 'ID' and 'c15' from model fitting
all_targets = [c for c in y_train.columns if c != "ID"]
cols_fit = [c for c in all_targets if c != "c15"]

# Create the prediction DataFrame (clip values to stay within [0,1])
pred_df = pd.DataFrame(np.clip(y_pred_blend, 0, 1), columns=cols_fit)

# Réinsère la cible manquante 'c15'
pred_df.insert(all_targets.index("c15"), "c15", 0.0)

# Assemble la soumission finale avec ID et ordre exact des colonnes
submission = pd.concat([X_test["ID"], pred_df[all_targets]], axis=1)
submission.to_csv("submission_ET_BLEND_v1.csv", index=False)


# Quick diagnostics
vals = submission.drop(columns=["ID"]).to_numpy()
assert np.isfinite(vals).all(), "NaN/Inf détectés dans la soumission"
assert (vals >= -1e-9).all() and (vals <= 1+1e-9).all(), "Valeurs hors [0,1]"
assert submission.index.equals(X_test.index)
print("✅ Submission prête :", submission.shape)
